In [1]:
import os
import csv

from sqlalchemy import create_engine, text
from sqlalchemy.engine.url import URL

In [2]:
DB_USERNAME = os.environ.get('DB_USERNAME')
DB_PASSWORD = os.environ.get('DB_PASSWORD')
DB_HOST = os.environ.get('DB_HOST')
DB_DATABASE = os.environ.get('DB_DATABASE')

In [3]:
url = URL.create(
        "mysql+mysqlconnector",
        username=DB_USERNAME,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=3306,
        database=DB_DATABASE,
    )
engine = create_engine(url)

In [4]:
with open('pst_20240426_google_ids.txt') as f:
    ids = f.read().splitlines()

In [18]:
def query_marc_records_from_db(engine, ids):
    results = []
    chunk_size = 500  # Adjust based on your database's performance and limits
    chunks = [ids[i:i + chunk_size] for i in range(0, len(ids), chunk_size)]
    with engine.connect() as connection:
        for chunk in chunks:
            # Generate the placeholders for each chunk
            placeholders = ','.join([':id{}'.format(i) for i in range(len(chunk))])
            
            # Construct the query with dynamic placeholders
            query_str = f"""select zr.id, zr.cid, zr.attr_ingest_date from zephir_records as zr 
                            where zr.id in ({placeholders})
                            and (select count(zr2.namespace) from zephir_records as zr2 where zr2.cid = zr.cid and zr2.attr_ingest_date is not null and zr2.namespace != 'PST') > 0;"""
            # Prepare parameters as a dictionary {placeholder_name: value}
            params = {f'id{i}': chunk[i] for i in range(len(chunk))}
            results.extend(connection.execute(text(query_str), params).fetchall())
    return results

In [19]:
result = query_marc_records_from_db(engine, ids)

In [20]:
len(result)

3566

In [21]:
with open("result.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(result)